# Paso 4.- Datos Holiday

## Determinar la variable de festividad de un día

Determinar la variable de festividad de un día: 
- laborables = 0
- Sábados = 0.75
- Domingos y festivos = 1


In [145]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from esiosdata import PVPC
from esiosdata.prettyprinting import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time


In [146]:
data_total = pd.read_csv("data_total.csv") 
data_total.head()

,Unnamed: 0,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_x,date_timestamp,date_y,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date.1,date_day,Brent_price
0,0,2015-10-01 00:00:00+02:00,117.77,69.49,71.88,2015-10-01 00:00:00+02:00,1.443650e+09,2015-10-01 00:00:00+02:00,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,2015-10-01 00:00:00+02:00,2015-10-01,47.48
1,1,2015-10-01 01:00:00+02:00,117.09,68.21,65.57,2015-10-01 01:00:00+02:00,1.443654e+09,2015-10-01 01:00:00+02:00,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,2015-10-01 01:00:00+02:00,2015-10-01,47.48
2,2,2015-10-01 02:00:00+02:00,114.59,65.34,62.53,2015-10-01 02:00:00+02:00,1.443658e+09,2015-10-01 02:00:00+02:00,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,2015-10-01 02:00:00+02:00,2015-10-01,47.48
3,3,2015-10-01 03:00:00+02:00,116.60,66.53,63.40,2015-10-01 03:00:00+02:00,1.443661e+09,2015-10-01 03:00:00+02:00,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,2015-10-01 03:00:00+02:00,2015-10-01,47.48
4,4,2015-10-01 04:00:00+02:00,122.07,71.03,67.37,2015-10-01 04:00:00+02:00,1.443665e+09,2015-10-01 04:00:00+02:00,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,2015-10-01 04:00:00+02:00,2015-10-01,47.48


In [147]:
#Some cleaning
data_total = data_total.loc[:, ~data_total.columns.str.contains('^Unnamed')]
data_total = data_total.drop(columns=['date_x', 'date_y', 'date.1'])
data_total['date_datetime'] = pd.to_datetime(data_total['fecha'])
data_total.head()

,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,date_datetime
0,2015-10-01 00:00:00+02:00,117.77,69.49,71.88,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,2015-10-01,47.48,2015-10-01 00:00:00+02:00
1,2015-10-01 01:00:00+02:00,117.09,68.21,65.57,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,2015-10-01,47.48,2015-10-01 01:00:00+02:00
2,2015-10-01 02:00:00+02:00,114.59,65.34,62.53,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,2015-10-01,47.48,2015-10-01 02:00:00+02:00
3,2015-10-01 03:00:00+02:00,116.60,66.53,63.40,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,2015-10-01,47.48,2015-10-01 03:00:00+02:00
4,2015-10-01 04:00:00+02:00,122.07,71.03,67.37,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,2015-10-01,47.48,2015-10-01 04:00:00+02:00


In [148]:
from pandas.tseries.holiday import Holiday,AbstractHolidayCalendar

from pandas.tseries.offsets import CustomBusinessDay

class EsBusinessCalendar(AbstractHolidayCalendar):
   rules = [
     Holiday('Año Nuevo', month=1, day=1, observance=nearest_workday),
     Holiday('Epifanía del Señor', month=1, day=6, observance=nearest_workday),
     Holiday('Viernes Santo', month=1, day=1, offset=[Easter(), Day(-2)]),
     Holiday('Día del Trabajador', month=5, day=1, observance=nearest_workday),
     Holiday('Asunción de la Virgen', month=8, day=15, observance=nearest_workday),
     Holiday('Día de la Hispanidad', month=10, day=12, observance=nearest_workday),
     Holiday('Todos los Santos', month=11, day=1, observance=nearest_workday),
     Holiday('Día Constitución', month=12, day=6, observance=nearest_workday),
     Holiday('Inmaculada Concepción', month=12, day=8, observance=nearest_workday),	    
     Holiday('Navidad', month=12, day=25, observance=nearest_workday)
   ]

cal = EsBusinessCalendar()
holidays = cal.holidays(start=data_total['fecha'].min(),
                        end=data_total['fecha'].max()).to_pydatetime()

holidays_array_by_date = list()
for holiday in holidays: 
    holidays_array_by_date.append(holiday.strftime('%Y-%m-%d'))
    
print(holidays_array_by_date)  

['2015-10-12', '2015-11-02', '2015-12-07', '2015-12-08', '2015-12-25', '2016-01-01', '2016-01-06', '2016-03-25', '2016-05-02', '2016-08-15', '2016-10-12', '2016-11-01', '2016-12-06', '2016-12-08', '2016-12-26', '2017-01-02', '2017-01-06', '2017-04-14', '2017-05-01', '2017-08-15', '2017-10-12', '2017-11-01', '2017-12-06', '2017-12-08', '2017-12-25', '2018-01-01', '2018-01-05', '2018-03-30', '2018-05-01', '2018-08-15', '2018-10-12', '2018-11-01', '2018-12-06', '2018-12-07', '2018-12-25', '2019-01-01', '2019-01-07', '2019-04-19', '2019-05-01']


In [149]:
from datetime import date

def is_holiday(df):
    for i, row in df.iterrows():
        holiday_value = 0
        if row['date_datetime'].weekday() == 5:
            holiday_value = 0.75
        elif row['date_datetime'].weekday() == 6:     
            holiday_value = 1 
        else:
             if row['date_datetime'] in holidays_array_by_date:
                holiday_value = 1
        df.set_value(i,'Holiday',holiday_value)        

is_holiday(data_total)
data_total = data_total.drop(columns=['date_datetime'])
data_total.tail()

/Users/davidcerezallanda/.conda/envs/practicaspy36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,Holiday
33822,2019-08-10 06:00:00+02:00,103.75,55.34,52.52,1.565410e+09,23617.000000,3466.666667,6913.500000,133.000000,18.000000,114.500000,452.833333,23642.666667,23749.0,2019-08-10,62.29,0.75
33823,2019-08-10 07:00:00+02:00,106.45,57.88,61.32,1.565413e+09,23759.666667,3234.500000,6925.833333,173.500000,63.500000,109.500000,452.166667,23771.333333,23837.0,2019-08-10,62.29,0.75
33824,2019-08-10 08:00:00+02:00,106.69,58.47,61.62,1.565417e+09,25005.833333,2508.166667,6928.166667,764.666667,655.500000,108.666667,453.666667,24975.333333,25119.0,2019-08-10,62.29,0.75
33825,2019-08-10 09:00:00+02:00,108.98,60.86,63.90,1.565420e+09,27365.333333,1702.833333,6927.166667,2211.166667,1849.833333,361.166667,456.500000,27397.666667,27387.0,2019-08-10,62.29,0.75
33826,2019-08-10 10:00:00+02:00,106.60,58.61,61.59,1.565424e+09,29265.400000,1284.200000,6926.600000,4463.800000,2886.000000,1577.400000,457.000000,29220.000000,29199.0,2019-08-10,62.29,0.75


In [150]:
data_total[data_total['Holiday'] == 1]

,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,Holiday
72,2015-10-04 00:00:00+02:00,102.60,55.91,56.94,1.443910e+09,23050.000000,2605.666667,4906.333333,36.000000,35.666667,0.000000,589.166667,22840.333333,22725.0,2015-10-04,46.55,1.0
73,2015-10-04 01:00:00+02:00,101.84,55.15,53.62,1.443913e+09,21724.333333,3071.333333,4903.833333,36.000000,35.666667,0.000000,587.833333,21529.166667,21149.0,2015-10-04,46.55,1.0
74,2015-10-04 02:00:00+02:00,100.45,53.41,51.72,1.443917e+09,20499.500000,3492.666667,4905.000000,36.000000,36.000000,0.000000,585.500000,20528.000000,20359.0,2015-10-04,46.55,1.0
75,2015-10-04 03:00:00+02:00,100.45,53.42,51.73,1.443920e+09,19972.833333,4090.833333,4903.000000,36.000000,36.000000,0.000000,585.833333,19854.333333,19538.0,2015-10-04,46.55,1.0
76,2015-10-04 04:00:00+02:00,98.72,51.69,50.01,1.443924e+09,19722.666667,4919.833333,4900.333333,36.000000,35.833333,0.000000,578.500000,19541.166667,19372.0,2015-10-04,46.55,1.0
77,2015-10-04 05:00:00+02:00,98.23,50.93,49.11,1.443928e+09,19497.000000,5869.166667,4904.000000,36.000000,35.666667,0.000000,574.500000,19557.000000,19096.0,2015-10-04,46.55,1.0
78,2015-10-04 06:00:00+02:00,100.92,52.55,50.26,1.443931e+09,19768.500000,6667.333333,4903.833333,36.000000,35.500000,0.166667,573.166667,19736.666667,19489.0,2015-10-04,46.55,1.0
79,2015-10-04 07:00:00+02:00,104.31,54.87,58.05,1.443935e+09,20401.000000,7162.833333,4902.000000,38.333333,37.500000,0.000000,573.833333,20411.666667,20211.0,2015-10-04,46.55,1.0
80,2015-10-04 08:00:00+02:00,108.71,58.33,62.35,1.443938e+09,20486.000000,7334.333333,4901.666667,85.000000,84.000000,0.000000,574.000000,20460.333333,20113.0,2015-10-04,46.55,1.0
81,2015-10-04 09:00:00+02:00,108.31,58.36,61.99,1.443942e+09,21944.333333,7628.166667,4901.000000,470.166667,464.833333,4.666667,575.666667,21873.166667,21572.0,2015-10-04,46.55,1.0


In [151]:
data_total.to_csv('data_total.csv')